In [2]:
import pandas as pd
df=pd.read_excel("C:/Users/32817/Desktop/all-2024-2025.xlsx")
df.head()

,数据来源,用户名,发帖时间,发帖内容,转发,评论,点赞,收藏,性别,发布地,粉丝数,关注数,总发微博数,转评赞
0,抑郁症超话,暗流0806,2025/04/30 10:39,有玩文明六的宝子吗？想有个人能学习一下或一起研究，因为很多地方会搞不懂,0,9,1,0,女,天津,110,210,103,211
1,抑郁症超话,Mirrollical,2025/04/30 22:27,都不知道为什么能吵起来，我说那个牙医诊所叫什么，我搜一下打电话问一下五一开不开门，非说直接去...,0,1,0,0,女,甘肃,62,548,983,12000
2,抑郁症超话,魔法少主一脸痴汉笑,2025/04/30 19:26,五一还要加班！！！！！！！！！还有谁！！！,0,1,0,0,女,浙江,4812,2969,23789,41000
3,抑郁症超话,隐僻的糖果店,2025/04/30 23:59,我不知道我是怎么了。一开始玩王者的时候，觉得这只是个游戏而已没必要太在乎输赢，开心就好，当时...,0,4,0,0,女,江西,49,11,29,161
4,抑郁症超话,HappyFeather鸟儿展翅飞向天空丨,2025/04/30 23:33,我真的好想死啊，我活不下去了，好痛苦,0,5,0,0,女,湖南,546,507,3691,20000


In [6]:
import pandas as pd
import re

# ======================
# 参数配置（根据实际数据调整）
# ======================
INPUT_FILE = "C:/Users/32817/Desktop/all-2024-2025.xlsx"    # 输入文件名
OUTPUT_FILE = "labeled_data.xlsx" # 输出文件名
TEXT_COLUMN = "发帖内容"          # 待处理文本列名

# 抑郁关键词
DEPRESSION_KEYWORDS = {
    '死', '崩溃', '撑不住', '麻木', '自残', '绝望', '自杀', '抑郁', '割腕',
    '跳楼', '想死', '活不下去', '阴湿', '女鬼', '霉运', '噩梦', '了结', '药',
    '医生', '复诊', '痛苦', '无力', '嘎', '鼠', '难过', '生病', '氟', '西汀',
    '舍曲林', '躯体化','孤独','悲伤','家庭','霸凌','睡眠','疲惫','为什么','意义','不适','糟糕','刀','烂','活不下去','病',
    '哭','难熬','失眠','住院','控制不住','敏感','爱我','愧疚','对不起','难受','别生','自虐','放弃','🔪','医院','住院',
    '委屈','劳拉西泮','活下去','佐匹克隆片','地西泮'
}

# 焦虑关键词
ANXIETY_KEYWORDS = {
    '焦虑', '心慌', '失眠', '压力', '害怕', '紧张', '发抖', '恐慌', '担心',
    '手抖', '喘不过气', '过度担心', '坐立不安', '烦躁', '骂人', '头晕', '呕吐',
    '呼吸', '喘', '抖', '燥热', '心跳', '疑病', '耳鸣', '躯体化','出汗','颤','无力','急促','无力','曲唑酮','刺痛',
    '氯硝西泮','吐','晕','焦慮','螺酮','劳拉','身體健康','伤痕','惊恐','堵','心率','心脏','帕罗','发作','舒必利',
    '肚子','肩胛','苯醌','酸镁','环酮','胃','身體','飘忽','恍惚','淋巴','阿普','肋骨'
}

# 混合关键词（触发抑郁焦虑标签）
MIXED_KEYWORDS = {'躁郁','崩溃','无助','焦躁','躯体化','重焦中郁','双向','躁期','躯体症状','发作'}

# 新增的关键词集（判定为 "其他" 类别）
OTHER_KEYWORDS = {'晚安', '新年快乐', '分享日常','除夕快乐','新的一年','超话粉丝大咖'}

def classify_text(text):
    """核心分类逻辑"""
    # ==== 新增：处理非字符串和空值 ====
    if not isinstance(text, str) or pd.isna(text):
        return "其他"
    
    # 清洗文本：去除超话标记和特殊符号
    cleaned = re.sub(r"抑郁症超话", "", text)
    cleaned = re.sub(r"\s+", "", cleaned.strip())
    
    # 空白内容判定
    if len(cleaned) <= 1 or cleaned in ["。", "..", "..."]:
        return "其他"
    
    # 关键词匹配
    has_depression = any(kw in cleaned for kw in DEPRESSION_KEYWORDS)
    has_anxiety = any(kw in cleaned for kw in ANXIETY_KEYWORDS)
    has_mixed = any(kw in cleaned for kw in MIXED_KEYWORDS)
    has_other = any(kw in cleaned for kw in OTHER_KEYWORDS)  # 新增的“其他”关键词
    
    # 优先判断混合情况：如果包含“抑郁”和“焦虑”的关键词
    if has_mixed or (has_depression and has_anxiety):
        return "抑郁焦虑"
    elif has_depression:
        return "抑郁"
    elif has_anxiety:
        return "焦虑"
    elif has_other:
        return "其他"  # 如果包含“其他”类关键词，则判定为 "其他"
    else:
        return "其他1"  # 如果没有匹配到任何已知关键词，归类为“其他”

# ======================
# 主处理流程
# ======================
if __name__ == "__main__":
    # 读取数据
    df = pd.read_excel(INPUT_FILE)
    
    # 执行分类
    df["auto_label"] = df[TEXT_COLUMN].apply(classify_text)
    
    # 标记是否需要人工复核（只对“其他”类文本进行复核）
    df["need_review"] = df["auto_label"] == "其他1"
    
    # 保存结果
    df.to_excel(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    print(f"处理完成！已保存到 {OUTPUT_FILE}")
    print("需要人工复核的数量：", df["need_review"].sum())


处理完成！已保存到 labeled_data.xlsx
需要人工复核的数量： 36216
